## Домашнее задание к лекции «Продвинутый pandas 2»

### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [ ]:
import pandas as pd

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.info()

In [ ]:
rat = ratings.groupby('userId').filter(lambda x: x['rating'].count() >= 100)
len(rat['userId'].unique())  #На всякий случай...

In [ ]:
rat.groupby('userId').agg({'timestamp': ['min', 'max']}).reset_index() #Это просто так...

In [ ]:
ltv = rat.groupby('userId')['timestamp'].apply(lambda x: x.max() - x.min()).mean()

In [ ]:
print('LTV для клиентов с более 100 рейтингами = \n', int(ltv/60), 'минут\n', int(ltv/3600), ' часов\n', int(ltv/3600/24), ' дней')

### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:  
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента 
аналогичную таблицу по типам выручки с указанием адреса клиента 
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [364]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [367]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [368]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [369]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [370]:
revenue = rzd.merge(auto, how = 'outer', on = 'client_id') \
            .merge(air, how = 'outer', on = 'client_id')   \
            .fillna(0)


In [371]:
revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [372]:
revenue_with_address = revenue.merge(client_base, how = 'outer', on = 'client_id')

In [373]:
revenue_with_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:  

* Известно ли время записи пары координат?

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

* Взять координаты магазина, рассчитать вектор расстояний от точки нахождения поьзователя до точки продажи.  Если расстояние совпадает (или в пределах погрешности), то фиксируем визит в магазин. Если коорждинаты в формате временного ряда, то в таблице фиксируется время визита, если нет, то количество визитов.  Дальше можно посчитать, за сколько визитов пользоваетль делает покупку, как часто он заходит в магазин и т.д.
*

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

* Да в приницпе никак...   Можно посмотреть распределение координат по плотности и понять, какие места пользователь посещает чаще всего, чтобы понять его предпочтения.  Это могут быть конкуренты, место его работы, жительства.  Из этого можно оценить перспективы открытия дополнительных точек продаж, лучше настроить таргет, ну и еще много всего... 
*

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

* Ну вот я выше ответил.  Наверное, на предыдущий вопрос надо было как то по другому отвечать... 